In [1]:
import json
import pandas as pd
import glob
from pathlib import Path
import networkx as nx
import matplotlib.pyplot as plt
import sys
import re
import collections
import os
import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

In [3]:
paths = [x for x in glob.glob('data_smaller/*')]

In [4]:
paths

['data_smaller/mpd.slice.376000-376999.csv',
 'data_smaller/mpd.slice.958000-958999.csv',
 'data_smaller/mpd.slice.341000-341999.csv',
 'data_smaller/mpd.slice.0-999.csv',
 'data_smaller/mpd.slice.290000-290999.csv',
 'data_smaller/mpd.slice.516000-516999.csv']

In [5]:
df = pd.concat(map(pd.read_csv, paths))

/tmp/ipykernel_112318/2300992062.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(map(pd.read_csv, paths))


In [113]:
df

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,playlist_pid,name,collaborative,modified_at,num_tracks,num_albums,num_followers,num_edits,duration_ms_playlist,num_artists,description
0,0,Chicago,spotify:track:0KMGxYKeUzK9wc5DZCt3HT,spotify:artist:3iDD7bnsjL9J4fO298r0L0,If You Leave Me Now - Remastered,spotify:album:4asBn7hzIq2Ka7rDzJN24P,235373,Chicago X,376000,Cheese,False,1490745600,95,78,2,15,25518141,41,NaN
1,1,Chicago,spotify:track:4OJFkrRQqol4FsPesF8eu4,spotify:artist:3iDD7bnsjL9J4fO298r0L0,Saturday In The Park - Remastered,spotify:album:2oSXXINsWGuEsc4udgWxh8,235666,Chicago V,376000,Cheese,False,1490745600,95,78,2,15,25518141,41,NaN
2,2,Chicago,spotify:track:7GqIDx2QVGOpd4r1fZaUUW,spotify:artist:3iDD7bnsjL9J4fO298r0L0,25 Or 6 To 4 - Remastered,spotify:album:0PRgsdDXQ8QPaDUetVF7yN,289800,Chicago II,376000,Cheese,False,1490745600,95,78,2,15,25518141,41,NaN
3,3,Chicago,spotify:track:07TnWCHrFkvF61GzJuLVt0,spotify:artist:3iDD7bnsjL9J4fO298r0L0,Hard To Say I'M Sorry / Get Away,spotify:album:4NmlDrnZOd3ejob9PoiOd9,304213,Chicago 16 [Expanded],376000,Cheese,False,1490745600,95,78,2,15,25518141,41,NaN
4,4,Chicago,spotify:track:36lkJSDyMEZoWTqtRj8Q8q,spotify:artist:3iDD7bnsjL9J4fO298r0L0,You're The Inspiration - 2006 Remastered Version,spotify:album:1ICKrl6sDjJD1YdR9VDfPR,229240,Chicago 17 [Expanded],376000,Cheese,False,1490745600,95,78,2,15,25518141,41,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373429,43,Childish Gambino,spotify:track:4r0GVpjSsKSR1biv4fOoa5,spotify:artist:73sIBHcqh3Z3NyqHKZ7FOL,Me and Your Mama,spotify:album:4xnq1L6P551Qcb9gBXNMK7,379226,"""Awaken, My Love!""",516999,monday,False,1496188800,48,40,1,6,11955349,32,NaN
373430,44,Fleetwood Mac,spotify:track:4bEb3KE4mSKlTFjtWJQBqO,spotify:artist:08GQAI4eElDnROBrJRGE0X,Don't Stop - Remastered,spotify:album:1bt6q2SruMsBtcerNVtpZB,193346,Rumours,516999,monday,False,1496188800,48,40,1,6,11955349,32,NaN
373431,45,Electric Light Orchestra,spotify:track:72ahyckBJfTigJCFCviVN7,spotify:artist:7jefIIksOi1EazgRTfW2Pk,Don't Bring Me Down,spotify:album:1CvVSn2MtKDBR6aWMkNkem,243373,Discovery,516999,monday,False,1496188800,48,40,1,6,11955349,32,NaN
373432,46,Grouplove,spotify:track:0GO8y8jQk1PkHzS31d699N,spotify:artist:3kVUvbeRdcrqQ3oHk5hPdx,Tongue Tied,spotify:album:3oylWMc9TTC6Nx4I6U3axc,218013,Never Trust A Happy Song,516999,monday,False,1496188800,48,40,1,6,11955349,32,NaN


In [21]:
# df.to_csv('data_for_neo4jadmin/combined_data.csv', index=None)

In [2]:
df = pd.read_csv('data_for_neo4jadmin/combined_data.csv')

In [3]:
df

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,playlist_pid,name,collaborative,modified_at,num_tracks,num_albums,num_followers,num_edits,duration_ms_playlist,num_artists,description
0,0,Chicago,spotify:track:0KMGxYKeUzK9wc5DZCt3HT,spotify:artist:3iDD7bnsjL9J4fO298r0L0,If You Leave Me Now - Remastered,spotify:album:4asBn7hzIq2Ka7rDzJN24P,235373,Chicago X,376000,Cheese,False,1490745600,95,78,2,15,25518141,41,NaN
1,1,Chicago,spotify:track:4OJFkrRQqol4FsPesF8eu4,spotify:artist:3iDD7bnsjL9J4fO298r0L0,Saturday In The Park - Remastered,spotify:album:2oSXXINsWGuEsc4udgWxh8,235666,Chicago V,376000,Cheese,False,1490745600,95,78,2,15,25518141,41,NaN
2,2,Chicago,spotify:track:7GqIDx2QVGOpd4r1fZaUUW,spotify:artist:3iDD7bnsjL9J4fO298r0L0,25 Or 6 To 4 - Remastered,spotify:album:0PRgsdDXQ8QPaDUetVF7yN,289800,Chicago II,376000,Cheese,False,1490745600,95,78,2,15,25518141,41,NaN
3,3,Chicago,spotify:track:07TnWCHrFkvF61GzJuLVt0,spotify:artist:3iDD7bnsjL9J4fO298r0L0,Hard To Say I'M Sorry / Get Away,spotify:album:4NmlDrnZOd3ejob9PoiOd9,304213,Chicago 16 [Expanded],376000,Cheese,False,1490745600,95,78,2,15,25518141,41,NaN
4,4,Chicago,spotify:track:36lkJSDyMEZoWTqtRj8Q8q,spotify:artist:3iDD7bnsjL9J4fO298r0L0,You're The Inspiration - 2006 Remastered Version,spotify:album:1ICKrl6sDjJD1YdR9VDfPR,229240,Chicago 17 [Expanded],376000,Cheese,False,1490745600,95,78,2,15,25518141,41,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373429,43,Childish Gambino,spotify:track:4r0GVpjSsKSR1biv4fOoa5,spotify:artist:73sIBHcqh3Z3NyqHKZ7FOL,Me and Your Mama,spotify:album:4xnq1L6P551Qcb9gBXNMK7,379226,"""Awaken, My Love!""",516999,monday,False,1496188800,48,40,1,6,11955349,32,NaN
373430,44,Fleetwood Mac,spotify:track:4bEb3KE4mSKlTFjtWJQBqO,spotify:artist:08GQAI4eElDnROBrJRGE0X,Don't Stop - Remastered,spotify:album:1bt6q2SruMsBtcerNVtpZB,193346,Rumours,516999,monday,False,1496188800,48,40,1,6,11955349,32,NaN
373431,45,Electric Light Orchestra,spotify:track:72ahyckBJfTigJCFCviVN7,spotify:artist:7jefIIksOi1EazgRTfW2Pk,Don't Bring Me Down,spotify:album:1CvVSn2MtKDBR6aWMkNkem,243373,Discovery,516999,monday,False,1496188800,48,40,1,6,11955349,32,NaN
373432,46,Grouplove,spotify:track:0GO8y8jQk1PkHzS31d699N,spotify:artist:3kVUvbeRdcrqQ3oHk5hPdx,Tongue Tied,spotify:album:3oylWMc9TTC6Nx4I6U3axc,218013,Never Trust A Happy Song,516999,monday,False,1496188800,48,40,1,6,11955349,32,NaN


In [4]:
df_track_node = df[['track_uri','track_name','duration_ms', 'artist_name', 'album_name']]

In [5]:
len(df_track_node['track_uri'].unique())

117311

In [6]:
df_track_node = df_track_node.drop_duplicates(subset=['track_uri'])

In [7]:
df_track_node['label'] = 'Track'

In [8]:
# df_track_node.to_csv('data_for_neo4jadmin/track_nodes.csv', index=None, header=False)

In [23]:
playlist_list = ["playlist_pid","name","collaborative", \
"modified_at","num_tracks","num_albums","num_followers","num_edits", \
"duration_ms_playlist","num_artists","description"]

In [31]:
df_playlist_node = df[playlist_list].drop_duplicates(subset=['playlist_pid'])

In [33]:
df_playlist_node['label'] = 'Playlist'

In [35]:
# df_playlist_node.to_csv('data_for_neo4jadmin/playlist_nodes.csv', index=None, header=False)

In [41]:
df_contains_track = df[["playlist_pid",'pos','track_uri']]

In [47]:
df_contains_track['type'] = 'CONTAIN_Track'

/tmp/ipykernel_112318/3667425078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_contains_track['type'] = 'CONTAIN_Track'


In [49]:
# df_contains_track.to_csv('data_for_neo4jadmin/contains_track.csv', index=None, header=False)

In [44]:
df_isin_playlist = df[["track_uri",'pos','playlist_pid']]

In [50]:
df_isin_playlist['type'] = 'ISIN_Playlist'

/tmp/ipykernel_112318/12342105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isin_playlist['type'] = 'ISIN_Playlist'


In [52]:
# df_isin_playlist.to_csv('data_for_neo4jadmin/isin_playlist.csv', index=None, header=False)

In [65]:
df_playlist_tem_1 = df[['playlist_pid', 'track_uri']]

In [66]:
df_playlist_tem_2 = df[['playlist_pid', 'track_uri']]

In [67]:
result = pd.merge(df_playlist_tem_1,df_playlist_tem_2, on=['track_uri'])

In [68]:
indexSame = result[ result['playlist_pid_x'] == result['playlist_pid_y'] ].index

In [69]:
result = result.drop(indexSame)

In [74]:
# k = []
# for i,r in  result.iterrows():
#     print ('here')
#     if r['playlist_pid_x'] == r['playlist_pid_y']:
#         print ('here')
#         k.append(i)

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


...

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


KeyboardInterrupt: 

In [83]:
value_for_matching = result[['playlist_pid_x',"playlist_pid_y"]]

In [85]:
matching_size = value_for_matching.groupby(value_for_matching.columns.tolist(),as_index=False).size()

In [98]:
matching_size = matching_size[['playlist_pid_x', 'size', "playlist_pid_y"]]

In [92]:
matching_size['label'] = "COMMON_TRACKS"

/tmp/ipykernel_4876/1253805117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_size['label'] = "COMMON_TRACKS"


In [93]:
# matching_size.to_csv('data_for_neo4jadmin/common_tracks.csv', index=None, header=False)

In [112]:
k = list(zip(list(zip(matching_size.values[:,0],matching_size.values[:,2])), matching_size.values[:,1]))

In [108]:
for i in k:
    if sorted(k[1])

,playlist_pid_x,size,playlist_pid_y
0,0,3,5
1,0,1,14
2,0,1,17
3,0,1,34
4,0,8,38
...,...,...,...
3914123,958999,1,958953
3914124,958999,2,958961
3914125,958999,2,958982
3914126,958999,1,958984
